In [1]:
from adc import align
import pandas as pd
import sys
import tifffile as tf
import dask.array as da
import numpy as np
import napari
from multiprocessing import Pool

In [2]:
centers = pd.read_csv('/home/aaristov/Anchor/Lena/Data/E.coli/Ciprofloxacin/20220526-MIC-e.coli-cipro/10cells/centers_bin16.csv')

In [3]:
template_bin16, mask_bin16 = tf.imread('/home/aaristov/Anchor/Lena/Data/E.coli/Ciprofloxacin/20220526-MIC-e.coli-cipro/10cells/bf_labels_bin16_v3.tif')

In [4]:
data_bin16 = da.from_zarr('/home/aaristov/Anchor/Lena/Data/E.coli/Ciprofloxacin/20220526-MIC-e.coli-cipro/10cells/day1/BF_TRITC_maxZ.zarr/3/')

In [5]:
data_bin2 = da.from_zarr('/home/aaristov/Anchor/Lena/Data/E.coli/Ciprofloxacin/20220526-MIC-e.coli-cipro/10cells/day1/BF_TRITC_maxZ.zarr/0/')

In [6]:
bf = data_bin16[4,0].compute()

In [7]:
align.calculate_padding(template_bin16.shape, bf.shape)

((18, 18), (22, 22))

In [49]:
%%time 
tvec_bin16 = align.reg.similarity(align.pad(template_bin16, bf.shape), bf, constraints=align.CONSTRAINTS)

CPU times: user 6.25 s, sys: 1.19 s, total: 7.44 s
Wall time: 7.44 s


In [51]:
tvec_bin16

{'tvec': array([-3.48556473, 10.7352447 ]),
 'success': 0.053013983869245856,
 'angle': -2.498352469932371,
 'scale': 1.0005251686341115,
 'Dscale': 0.0005542525572911843,
 'Dangle': 0.01596877217885025,
 'Dt': 0.25,
 'timg': array([[21841.04753565, 21840.49966364, 21840.0966692 , ...,
         21843.83983972, 21842.67423951, 21841.76135565],
        [21841.09616634, 21840.60920702, 21840.25732524, ...,
         21843.57776352, 21842.55235012, 21841.73610763],
        [21841.107017  , 21840.68982185, 21840.39515843, ...,
         21843.24704731, 21842.36894809, 21841.66141204],
        ...,
        [21840.51938268, 21839.89775951, 21839.41617551, ...,
         21844.44981844, 21841.91744543, 21841.42925704],
        [21840.77309128, 21840.1527761 , 21839.68149688, ...,
         21844.64189466, 21842.31339231, 21841.65016562],
        [21840.94515608, 21840.35092941, 21839.90684984, ...,
         21845.92695882, 21841.99155267, 21841.95560514]])}

In [8]:
def rot(vec:np.ndarray, angle_deg:float, canvas_shape=(0,0)):
    theta = angle_deg / 180. * np.pi
    pivot = np.array(canvas_shape) / 2.
        
    c = np.cos(theta)
    s = np.sin(theta)
    mat = np.array([[c, -s], [s,c]])
    
    return np.dot(vec - pivot, mat) + pivot

def trans(vec:np.ndarray, tr_vec:np.ndarray):
    return vec + tr_vec

In [9]:
yx = centers[["y","x"]].values

In [10]:
ccenters  = yx - np.array(template_bin16.shape) / 2.

In [11]:
def move_centers(centers, tvec:dict, figure_size):
    return rot(
        centers / tvec["scale"] ,
        tvec["angle"],
    ) - tvec["tvec"] + np.array(figure_size) / 2.

In [12]:
align.CONSTRAINTS

{'scale': [1, 0.2], 'tx': [0, 50], 'ty': [0, 50], 'angle': [0, 30]}

In [13]:
def locate_wells(bf, template, ccenters):
    tvec = align.reg.similarity(align.pad(template, bf.shape), bf, constraints={'scale': [1, 0.1], 'tx': [0, 50], 'ty': [0, 50], 'angle': [0, 3]})  
    print(tvec)
    return move_centers(ccenters, tvec, bf.shape)

In [15]:
bf_data = data_bin16[:,0].compute()

In [16]:
%%time
def func(bf):
    return locate_wells(bf, template_bin16, ccenters)

p = Pool(6)
try:
    centers16 = p.map(func, bf_data)
except Exception as e:
    print(e)
finally:
    p.close()

{'tvec': array([ 7.41521828, -5.01025831]), 'success': 0.9492813713032735, 'angle': 0.3250219589967003, 'scale': 1.000010029846673, 'Dscale': 0.0005539671901667496, 'Dangle': 0.01596877217885025, 'Dt': 0.25}
{'tvec': array([15.19319745, 12.73440021]), 'success': 0.03744237556702879, 'angle': 0.3009538165028687, 'scale': 1.000338782849441, 'Dscale': 0.0005541493067173508, 'Dangle': 0.01596877217885025, 'Dt': 0.25}
{'tvec': array([ 20.73566667, -10.33345991]), 'success': 0.05417756398257824, 'angle': -0.007623382186125127, 'scale': 1.0013273253291166, 'Dscale': 0.0005546969213246866, 'Dangle': 0.01596877217885025, 'Dt': 0.25}
{'tvec': array([-3.477736  , 10.72915502]), 'success': 0.05280478933289002, 'angle': -2.4952416737408782, 'scale': 1.0005264830720422, 'Dscale': 0.0005542532854393682, 'Dangle': 0.01596877217885025, 'Dt': 0.25}
{'tvec': array([ 22.26023391, -14.98781366]), 'success': 0.044481804667993695, 'angle': 0.7751750895648968, 'scale': 1.0006719708254652, 'Dscale': 0.00055433

In [17]:
def add_new_dim(centers, value):
    return np.concatenate((np.ones((len(centers),1)) * value, centers), axis=1)

In [18]:
sample = np.concatenate([add_new_dim(c*8, i) for i,c in enumerate(centers16)])

In [19]:
v = napari.Viewer()

In [20]:
v.add_points(sample
        , size=300, face_color="#00000000", edge_color="#88000088")

<Points layer 'sample' at 0x7f51503f32e0>

reading /home/aaristov/Anchor/Lena/Data/E.coli/Ciprofloxacin/20220526-MIC-e.coli-cipro/10cells/day1/BF_TRITC_maxZ.zarr
found channel axis 1


In [21]:
pd.DataFrame(sample, columns=('sample', 'y', 'x')).to_csv('/home/aaristov/Anchor/Lena/Data/E.coli/Ciprofloxacin/20220526-MIC-e.coli-cipro/10cells/day1/BF_TRITC_maxZ.zarr/.centers.csv', index='sample')

In [79]:
v.add_points(move_centers(ccenters, tvec_bin16, bf.shape) * 8, size=100)

<Points layer 'Points [1]' at 0x7f50af356d90>

In [62]:
v.add_points(yx)

<Points layer 'yx' at 0x7f50af11d280>

In [17]:
v.add_image(tvec_bin16["timg"])

<Image layer 'Image' at 0x7f50afff2310>

In [44]:
v.add_image(bf)

<Image layer 'bf' at 0x7f50a4ba1af0>

In [77]:
path = '/home/aaristov/Anchor/Lena/Data/E.coli/Ciprofloxacin/20220526-MIC-e.coli-cipro/10cells/day1/BF_TRITC_maxZ.zarr/{}/'
v.add_image([da.from_zarr(path.format(i)) for i in range(4)])



<Image layer 'Image' at 0x7f5075e42e80>

In [63]:
v.add_image(tvec_bin16["timg"], name='transformed')

<Image layer 'transformed' at 0x7f50af2c9730>